In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.patches as mpatches
import numpy as np

cities_df = pd.read_csv('../data/cities.csv')
lines_df = pd.read_csv('../data/lines.csv')
station_lines_df = pd.read_csv('../data/station_lines.csv')
stations_df = pd.read_csv('../data/stations.csv')
systems_df= pd.read_csv('../data/systems.csv')
track_lines_df = pd.read_csv('../data/track_lines.csv')
track_df = pd.read_csv('../data/tracks.csv')



In [ ]:

cities_df.head()
cities_df["Long"] = cities_df["coords"].apply(lambda x: x.split('POINT(')[1].split(' ')[0])
cities_df["Lat"] = cities_df["coords"].apply(lambda x: x.split('POINT(')[1].split(' ')[1].split(")")[0])
cities_df = cities_df.drop(["country_state", "coords"], axis = 1)
cities_df

cities_df["start_year"] = cities_df["start_year"].fillna(0)
cities_df.info()

fig = px.scatter_geo(cities_df, 
                     lat='Lat', 
                     lon='Long',
                     hover_name='name', 
                     projection='natural earth',
                     title=' Map')
fig.show()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          334 non-null    int64  
 1   name        334 non-null    object 
 2   start_year  334 non-null    float64
 3   url_name    334 non-null    object 
 4   country     334 non-null    object 
 5   Long        334 non-null    object 
 6   Lat         334 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 18.4+ KB


In [ ]:
lines_df.info()
lines_df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1343 entries, 0 to 1342
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1343 non-null   int64  
 1   city_id            1343 non-null   int64  
 2   name               1343 non-null   object 
 3   url_name           1343 non-null   object 
 4   color              1343 non-null   object 
 5   system_id          1343 non-null   int64  
 6   transport_mode_id  926 non-null    float64
dtypes: float64(1), int64(3), object(3)
memory usage: 73.6+ KB


,id,city_id,name,url_name,color,system_id,transport_mode_id
124,118,69,Waterloo & City Line,118-waterloo-&-city-line,#50e3c2,259,4.0
125,115,69,Northern Line,115-norther-line,#000,259,4.0
126,117,69,"Circle Line - Circle via Euston, Victoria, Tow...",117-circle-line,#fdfa07,259,3.0
147,116,69,Central Line - Epping to West Ruislip & Ealing...,116-central-line,#fb0303,259,4.0
245,143,69,Victoria Line,143-victoria,#02dafb,259,4.0
249,162,69,Piccadilly Line,162-piccadilly-line,#000bbc,259,4.0
258,203,69,Tramlink,203-tramlink,#50e71b,263,9.0
259,167,69,Emirates Airline,167-emirates-airline,#c00000,263,7.0
260,209,69,Elizabeth Line,209-elizabeth-line,#6518a1,263,3.0
282,160,69,DLR,160-dlr,#86e9c8,263,5.0


In [4]:
station_lines_df.info()
station_lines_df.sample(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16242 entries, 0 to 16241
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          16242 non-null  int64 
 1   station_id  16242 non-null  int64 
 2   line_id     16242 non-null  int64 
 3   city_id     16242 non-null  int64 
 4   created_at  16242 non-null  object
 5   updated_at  16242 non-null  object
dtypes: int64(4), object(2)
memory usage: 761.5+ KB


,id,station_id,line_id,city_id,created_at,updated_at
10619,14789,14492,1338,206,2018-10-11 19:09:31.521099,2018-10-11 19:09:31.521099
5434,5580,256,70,4,2017-11-21 00:00:00,2017-11-21 00:00:00
6537,15901,15591,1475,91,2018-11-11 21:35:32.684827,2018-11-11 21:35:32.684827
5914,6063,5267,362,95,2017-11-21 00:00:00,2017-11-21 00:00:00
16220,16773,16537,1603,91,2019-03-08 22:28:54.827849,2019-03-08 22:28:54.827849
223,281,6595,564,114,2017-11-21 00:00:00,2017-11-21 00:00:00
9865,12039,12068,1105,1,2018-07-08 00:50:22.778778,2018-07-08 00:50:22.778778
1790,1828,7342,615,23,2017-11-21 00:00:00,2017-11-21 00:00:00
9620,11038,3415,271,106,2018-04-04 19:32:18.399729,2018-04-04 19:32:18.399729
15832,16315,15996,1500,91,2019-01-03 22:21:15.10247,2019-01-03 22:21:15.10247


In [5]:
stations_df.info()
stations_df.head(10)
stations_df[stations_df["city_id"]==69]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15794 entries, 0 to 15793
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          15794 non-null  int64  
 1   name        14248 non-null  object 
 2   geometry    15794 non-null  object 
 3   buildstart  14342 non-null  float64
 4   opening     15721 non-null  float64
 5   closure     15713 non-null  float64
 6   city_id     15794 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 863.9+ KB


,id,name,geometry,buildstart,opening,closure,city_id
528,940,NaN,POINT(-0.0498506485737096 51.4982818582583),1993.0,1999.0,999999.0,69
738,1226,NaN,POINT(-0.0605422158858456 51.5195940456462),1880.0,1884.0,999999.0,69
817,1204,NaN,POINT(-0.284869015664782 51.4770067389025),1875.0,1877.0,999999.0,69
842,1205,NaN,POINT(-0.301248558553624 51.4630848598107),1875.0,1877.0,999999.0,69
852,1299,North Quay,POINT(-0.0206325056204832 51.506833669525),1985.0,1987.0,999999.0,69
...,...,...,...,...,...,...,...
15498,16226,Sundridge Hill,POINT(0.481842244838987 51.385505610752),1860.0,1860.0,999999.0,69
15499,16227,Lower Higham,POINT(0.467832135765462 51.4255464342704),1846.0,1849.0,999999.0,69
15500,16228,Benfleet,POINT(0.561452273618102 51.5442670989896),1852.0,1855.0,999999.0,69
15501,16229,Broxbourne,POINT(-0.0110359029796427 51.7468567786846),1839.0,1840.0,999999.0,69


In [6]:
systems_df.info()
systems_df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488 entries, 0 to 487
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       488 non-null    int64 
 1   city_id  488 non-null    int64 
 2   name     287 non-null    object
dtypes: int64(2), object(1)
memory usage: 11.6+ KB


,id,city_id,name
0,1,5,NaN
1,2,6,NaN
2,3,7,NaN
3,4,8,NaN
4,5,9,NaN
5,6,16,NaN
6,7,10,NaN
7,8,11,NaN
8,9,12,NaN
9,10,13,NaN


In [7]:
track_lines_df.info()
track_lines_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9757 entries, 0 to 9756
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          9757 non-null   int64 
 1   section_id  9757 non-null   int64 
 2   line_id     9757 non-null   int64 
 3   created_at  9757 non-null   object
 4   updated_at  9757 non-null   object
 5   city_id     9757 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 457.5+ KB


,id,section_id,line_id,created_at,updated_at,city_id
0,2494,1278,343,2017-11-21 00:00:00,2017-11-21 00:00:00,252
1,4124,4477,779,2017-11-21 00:09:55.135507,2017-11-21 00:09:55.135507,63
2,2495,21,9,2017-11-21 00:00:00,2017-11-21 00:00:00,1
3,2496,940,228,2017-11-21 00:00:00,2017-11-21 00:00:00,79
4,4129,4478,793,2017-11-21 17:44:39.765832,2017-11-21 17:44:39.765832,48
5,4139,4483,797,2017-11-23 21:51:04.515237,2017-11-23 21:51:04.515237,114
6,4180,4522,657,2017-11-30 17:36:52.150194,2017-11-30 17:36:52.150194,1
7,4181,4523,657,2017-11-30 18:02:36.400933,2017-11-30 18:02:36.400933,1
8,4182,4524,657,2017-11-30 18:02:36.429198,2017-11-30 18:02:36.429198,1
9,4184,4526,657,2017-11-30 18:50:01.420808,2017-11-30 18:50:01.420808,1


In [8]:
track_df.info()
track_df.head(10)
track_df[track_df["closure"] < 999999]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9271 entries, 0 to 9270
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          9271 non-null   int64  
 1   geometry    9271 non-null   object 
 2   buildstart  7663 non-null   float64
 3   opening     9250 non-null   float64
 4   closure     9240 non-null   float64
 5   length      9271 non-null   int64  
 6   city_id     9271 non-null   int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 507.1+ KB


,id,geometry,buildstart,opening,closure,length,city_id
26,3891,"LINESTRING(2.54098090855223 48.8401001862531,2...",1991.0,1993.0,1993.0,753,95
27,3920,"LINESTRING(139.711149052338 35.6490980321521,1...",1924.0,1924.0,1969.0,1099,114
36,4893,"LINESTRING(-87.6337015470323 41.8755914840214,...",1895.0,1895.0,1958.0,9451,147
39,4894,"LINESTRING(-87.7449583 41.8725646,-87.7548059 ...",1905.0,1905.0,1958.0,818,147
52,4896,"LINESTRING(-87.8827459601093 41.880327150266,-...",1926.0,1926.0,1951.0,3515,147
...,...,...,...,...,...,...,...
9174,9419,"LINESTRING(135.18459 34.24191,135.18571 34.241...",1917.0,1917.0,1924.0,1672,91
9176,9414,"LINESTRING(-58.3618 -34.67812,-58.36194 -34.67...",1969.0,1969.0,1977.0,5737,1
9177,9415,"LINESTRING(-58.39615 -34.66671,-58.39619 -34.6...",1969.0,1969.0,1977.0,3083,1
9186,9490,"LINESTRING(-58.45003 -34.55816,-58.4505 -34.55...",1857.0,1863.0,2019.0,925,1
